# Model creation and data split-up

This notebook will:
* Split data ito test and train data
* Train the model
* Save the model

Import packages

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pds

Import data

In [2]:
df_tr = pds.read_pickle('ANN_train_203_downsampled.pkl')
df_te = pds.read_pickle('ANN_test_203_downsampled.pkl')
df = pds.concat([df_tr, df_te], keys=['train', 'test'])
df.head()

direction  journey        lat        lon  seg  speed  \
train 0  129.852944        1  15.571012  58.414258    1   0.00   
      1  142.963046        1  15.571246  58.414280    1   3.08   
      2  287.399994        1  15.571079  58.414436    1   4.37   
      3  289.790009        1  15.570891  58.414459    1   6.73   
      4  288.200012        1  15.570773  58.414478    1   7.30   

                        timestamp  time_left  segment_time  tsjs    lat_one  \
train 0 2018-02-16 04:48:46+01:00       65.0          65.0   0.0  15.571012   
      1 2018-02-16 04:49:05+01:00       46.0          65.0   0.0  15.571012   
      2 2018-02-16 04:49:17+01:00       34.0          65.0   0.0  15.571246   
      3 2018-02-16 04:49:18+01:00       33.0          65.0   0.0  15.571079   
      4 2018-02-16 04:49:19+01:00       32.0          65.0   0.0  15.570891   

           lon_one    lat_two    lon_two  lat_three  lon_three  speed_one  \
train 0  58.414258  15.571012  58.414258  15.571012  58.414258  15.571012   
      1  58.414258  15.571012  58.414258  15.571012  58.414258  15.571012   
      2  58.414280  15.571012  58.414258  15.571012  58.414258  15.571246   
      3  58.414436  15.571246  58.414280  15.571012  58.414258  15.571079   
      4  58.414459  15.571079  58.414436  15.571246  58.414280  15.570891   

         speed_two  speed_three  
train 0  15.571012    15.571012  
      1  15.571012    15.571012  
      2  15.571012    15.571012  
      3  15.571246    15.571012  
      4  15.571079    15.571246

In [3]:
#df = pds.read_pickle('ANN_dataset.pkl')
#df.head()

Note that `time_left` is the label in this model 

In [4]:
label_string = 'time_left'

One hot encode segments, normalize columns and create final dataset

In [5]:
# Make segments categorical
dataset = pds.get_dummies(df['seg'])

# Combine segments, timestamps and time from journey start

# Cyclical timestamps 
dataset['hr_sin'] = np.sin(df.timestamp.dt.hour*(2.*np.pi/24))
dataset['hr_cos'] = np.cos(df.timestamp.dt.hour*(2.*np.pi/24))

# Convert to radians befor trigonometric functions
dataset['dir_sin'] = np.sin(df.direction*(np.pi/180))
dataset['dir_cos'] = np.cos(df.direction*(np.pi/180))

dataset['speed'] = df['speed']
dataset['tsjs'] = (df['tsjs']-df['tsjs'].min())/(df['tsjs'].max()-df['tsjs'].min())

dataset['lat'] = df['lat']
dataset['lon'] = df['lon']

# Include journey number to select entire journeys
dataset['journey'] = df['journey']

dataset.head()

1  2  3  4  5  6  7  8  9  10  11    hr_sin  hr_cos   dir_sin  \
train 0  1  0  0  0  0  0  0  0  0   0   0  0.866025     0.5  0.767692   
      1  1  0  0  0  0  0  0  0  0   0   0  0.866025     0.5  0.602330   
      2  1  0  0  0  0  0  0  0  0   0   0  0.866025     0.5 -0.954240   
      3  1  0  0  0  0  0  0  0  0   0   0  0.866025     0.5 -0.940940   
      4  1  0  0  0  0  0  0  0  0   0   0  0.866025     0.5 -0.949972   

          dir_cos     speed  tsjs       lat       lon  journey  
train 0 -0.640819  0.051520   0.0  0.325030  0.926504        1  
      1 -0.798247  0.210201   0.0  0.329938  0.928719        1  
      2  0.299041  0.276662   0.0  0.326437  0.944444        1  
      3  0.338574  0.398248   0.0  0.322491  0.946743        1  
      4  0.312335  0.427615   0.0  0.320008  0.948659        1

Select ~20% test journeys

divider = 0.2
num_journeys = dataset.journey.unique()[-1]
num_test_segments = np.int_(np.round(num_journeys*divider))
msk = np.random.randint(1, num_journeys, num_test_segments)
mask = dataset['journey'].isin(msk)

Divide training and test data, where data is a sequence from a journey

train_input = dataset[~mask]
test_input = dataset[mask]

train_labels = df[label_string][~mask]
test_labels = df[label_string][mask]

test_idxs = test_input.index

In [6]:
train_input = dataset.loc['train']
test_input = dataset.loc['test']

train_labels = df.loc['train'][label_string]
test_labels = df.loc['test'][label_string]

test_idxs = test_labels.index

In [7]:
# Journey numbers were needed up until this point
train_data = train_input.drop(columns=['journey'])
test_data = test_input.drop(columns=['journey'])

Build model

In [8]:
model = keras.Sequential([
	keras.layers.Dense(2*len(train_data.columns), activation=tf.nn.relu, input_shape=(train_data.shape[1],)),
    keras.layers.Dense(1*len(train_data.columns), activation=tf.nn.tanh),
    keras.layers.Dense(1*len(train_data.columns)),
	keras.layers.Dense(1, activation=tf.nn.relu)
	])

optimizer = keras.optimizers.Adadelta()
model.compile(loss='mae', optimizer = optimizer, metrics=['mae'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 38)                760       
_________________________________________________________________
dense_1 (Dense)              (None, 19)                741       
_________________________________________________________________
dense_2 (Dense)              (None, 19)                380       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 20        
Total params: 1,901
Trainable params: 1,901
Non-trainable params: 0
_________________________________________________________________
None


Fit model, 10% of the training data is used as validation set

In [9]:
EPOCHS=10
result = model.fit(train_data.values, 
                   train_labels.values, 
                   epochs = EPOCHS, 
                   validation_split=0.1, 
                   initial_epoch=0,
                   batch_size=32
                  )

Train on 1354825 samples, validate on 150537 samples
Epoch 1/10
1354825/1354825 [==============================] - 42s 31us/step - loss: 3.9298 - mean_absolute_error: 3.9298 - val_loss: 3.7431 - val_mean_absolute_error: 3.7431
Epoch 2/10
1354825/1354825 [==============================] - 41s 30us/step - loss: 3.5004 - mean_absolute_error: 3.5004 - val_loss: 3.7899 - val_mean_absolute_error: 3.7899
Epoch 3/10
1354825/1354825 [==============================] - 40s 30us/step - loss: 3.4259 - mean_absolute_error: 3.4259 - val_loss: 3.6352 - val_mean_absolute_error: 3.6352
Epoch 4/10
1354825/1354825 [==============================] - 41s 30us/step - loss: 3.3811 - mean_absolute_error: 3.3811 - val_loss: 3.5346 - val_mean_absolute_error: 3.5346
Epoch 5/10
1354825/1354825 [==============================] - 41s 30us/step - loss: 3.3493 - mean_absolute_error: 3.3493 - val_loss: 3.5159 - val_mean_absolute_error: 3.5159
Epoch 6/10
1354825/1354825 [==============================] - 41s 30us/step -

Evaluate model on test data

In [10]:
test_data.head()

,1,2,3,4,5,6,7,8,9,10,11,hr_sin,hr_cos,dir_sin,dir_cos,speed,tsjs,lat,lon
0,1,0,0,0,0,0,0,0,0,0,0,0.965926,-0.258819,0.653850,-0.756624,0.199897,0.0,0.325446,0.920498
1,1,0,0,0,0,0,0,0,0,0,0,0.965926,-0.258819,0.530770,-0.847516,0.250902,0.0,0.331089,0.926317
2,1,0,0,0,0,0,0,0,0,0,0,0.965926,-0.258819,-0.960828,0.277147,0.289026,0.0,0.327078,0.940613
3,1,0,0,0,0,0,0,0,0,0,0,0.965926,-0.258819,-0.964095,0.265556,0.329212,0.0,0.325295,0.941762
4,1,0,0,0,0,0,0,0,0,0,0,0.965926,-0.258819,-0.947713,0.319124,0.371458,0.0,0.323273,0.943295


In [11]:
model.evaluate(test_data.values, test_labels.values)

375276/375276 [==============================] - 4s 11us/step


[3.280032230226053, 3.280032230226053]

Prepare data for the evaluation step

In [12]:
predictions = model.predict(test_data.values)

### Create dataset for evaluation

In [13]:
segment = df['seg'].loc['test'].reset_index(drop=True)

In [14]:
output = pds.DataFrame()
output['seg'] = segment
output['journey'] = test_input['journey'].values
output['speed'] = test_data.speed.values
output['pred'] = predictions
output['label'] = test_labels.values
output.head()

,seg,journey,speed,pred,label
0,1,12,0.199897,53.656322,55.0
1,1,12,0.250902,49.445297,46.0
2,1,12,0.289026,38.639950,37.0
3,1,12,0.329212,37.714104,36.0
4,1,12,0.371458,36.436054,35.0


In [15]:
# Same name for all files
model_name = 'ANN_M2_downsampled'

# serialize model to YAML
model_yaml = model.to_yaml()
with open(model_name + '.yaml', 'w') as yaml_file:
    yaml_file.write(model_yaml)

# Serialize weights to HDF5
model.save_weights(model_name + '.h5')

In [16]:
output.to_pickle(model_name + '.pkl')

Done! Get ready for evaluation